In [1]:
import os
import requests
import dotenv
import time
import numpy as np
import pandas as pd
import alpaca_trade_api as alpaca_api
import krakenex
import pykrakenapi

%matplotlib inline

In [2]:
def random_portfolios_generator(tickers, n):
    random_portfolios = pd.DataFrame({'ticker':tickers})
    for n in range(0, n):
        random_portfolios[f'random weights {n+1}'] = [np.random.rand() for i in range (0, len(tickers))]
        random_portfolios[f'random weights {n+1}'] = random_portfolios[f'random weights {n+1}'] / sum(random_portfolios[f'random weights {n+1}'])
    return(random_portfolios)

In [3]:
dotenv.load_dotenv()

True

In [4]:
alpaca_api_key = os.getenv('alpaca_api_key')
alpaca_secret_key = os.getenv('alpaca_secret_key')
kraken_api_key = os.getenv('kraken_api_key')
kraken_secret_key = os.getenv('kraken_secret_key')

print(type(alpaca_api_key))
print(type(alpaca_secret_key))
print(type(kraken_api_key))
print(type(kraken_secret_key))

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>


In [5]:
alpaca = alpaca_api.REST(alpaca_api_key, alpaca_secret_key, api_version="v2")

faangmula_tickers = ['AAPL', 'ABNB', 'AMZN', 'FB', 'GOOG', 'LYFT', 'MSFT', 'NFLX', 'UBER']
start = pd.Timestamp('2015-12-01', tz='America/New_York').isoformat()

faangmula_original = alpaca.get_bars(faangmula_tickers, alpaca_api.TimeFrame.Day, start = start).df
faangmula_original

,open,high,low,close,volume,trade_count,vwap,symbol
timestamp,,,,,,,,
2015-12-01 05:00:00+00:00,118.73,118.810,116.860,117.34,34852374,187129,117.756760,AAPL
2015-12-02 05:00:00+00:00,117.00,118.110,116.080,116.28,33385643,180616,117.151198,AAPL
2015-12-03 05:00:00+00:00,116.55,116.790,114.220,115.20,41560785,245330,115.434888,AAPL
2015-12-04 05:00:00+00:00,115.20,119.250,115.110,119.03,57776977,307788,118.187290,AAPL
2015-12-07 05:00:00+00:00,118.98,119.860,117.810,118.28,32080754,190809,118.509111,AAPL
...,...,...,...,...,...,...,...,...
2022-04-04 04:00:00+00:00,36.40,36.930,36.215,36.51,20197208,146275,36.621379,UBER
2022-04-05 04:00:00+00:00,36.43,36.610,34.720,34.96,20987999,153337,35.188058,UBER
2022-04-06 04:00:00+00:00,33.82,34.000,32.550,33.41,29163174,203021,33.287397,UBER


In [6]:
faangmula = faangmula_original.drop(['open', 'high', 'low', 'volume', 'trade_count', 'vwap'], axis = 1).pivot(columns = 'symbol').fillna(0.0)
faangmula.index = faangmula.index.date
faangmula.columns = faangmula_tickers
faangmula

,AAPL,ABNB,AMZN,FB,GOOG,LYFT,MSFT,NFLX,UBER
2015-12-01,117.34,0.00,679.06,107.12,767.04,0.00,55.22,125.37,0.00
2015-12-02,116.28,0.00,676.01,106.07,762.38,0.00,55.21,128.93,0.00
2015-12-03,115.20,0.00,666.25,104.38,752.54,0.00,54.20,126.81,0.00
2015-12-04,119.03,0.00,672.64,106.18,766.81,0.00,55.91,130.93,0.00
2015-12-07,118.28,0.00,669.83,105.61,763.25,0.00,55.81,125.36,0.00
...,...,...,...,...,...,...,...,...,...
2022-04-04,178.44,177.02,3366.93,233.89,2872.85,38.96,314.97,391.50,36.51
2022-04-05,175.06,171.21,3281.10,231.84,2821.26,37.15,310.88,380.15,34.96
2022-04-06,171.59,164.66,3175.12,223.30,2743.52,36.08,299.50,368.35,33.41
2022-04-07,172.14,165.91,3155.69,222.95,2729.30,34.61,301.37,362.15,32.27


In [7]:

kraken = krakenex.API(kraken_secret_key, kraken_api_key)

crypto_tickers = ['BTC','ETH','LUNA','SOL','XRP','ADA',
    'AVAX','DOT','DOGE','SHIB','MATIC',
    'DAI','LTC','ATOM','LINK','UNI','TRX','BCH']

pykraken = pykrakenapi.KrakenAPI(kraken)

crypto_original = pd.DataFrame()
for ticker in crypto_tickers:
    ticker_ohlc, last = pykraken.get_ohlc_data((ticker + 'USD'), 1440, 1554597829)
    ticker_ohlc['symbol'] = ticker.replace('/USD', '')
    crypto_original = pd.concat([ticker_ohlc, crypto_original], axis=0)
    time.sleep(1)

In [8]:
crypto = crypto_original.drop(['time', 'open', 'high', 'low', 'vwap', 'volume', 'count'], axis = 1).pivot(columns = 'symbol').fillna(0.0)
crypto.index = crypto.index.date
crypto.columns = crypto_tickers
crypto

,BTC,ETH,LUNA,SOL,XRP,ADA,AVAX,DOT,DOGE,SHIB,MATIC,DAI,LTC,ATOM,LINK,UNI,TRX,BCH
2020-04-20,0.033990,2.2787,0.00,219.10,6839.2,1.01453,0.001960,0.0000,170.49,3.44413,40.32,0.00,0.0000,0.000000,0.00,0.012446,0.000,0.18177
2020-04-21,0.034635,2.2768,0.00,219.90,6852.1,1.01021,0.001984,0.0000,170.99,3.46270,40.65,0.00,0.0000,0.000000,0.00,0.012606,0.000,0.18340
2020-04-22,0.036665,2.3895,0.00,233.60,7136.9,1.01501,0.002005,0.0000,183.16,3.68806,41.83,0.00,0.0000,0.000000,0.00,0.012969,0.000,0.18815
2020-04-23,0.040810,2.4300,0.00,237.50,7485.0,1.01643,0.002060,0.0000,185.68,3.73123,42.95,0.00,0.0000,0.000000,0.00,0.013550,0.000,0.19283
2020-04-24,0.041764,2.7538,0.00,237.90,7507.8,1.01093,0.002084,0.0000,187.72,3.78000,44.51,0.00,0.0000,0.000000,0.00,0.013920,0.000,0.19328
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-05,1.170137,29.2892,91.02,365.47,45479.2,0.99985,0.172689,21.7782,3407.70,16.74518,122.79,116.27,1.5938,0.000027,126.82,0.069272,11.122,0.81726
2022-04-06,1.053412,26.9086,82.81,331.19,43179.4,0.99987,0.143326,19.8483,3168.44,15.40933,111.79,108.10,1.4550,0.000024,113.02,0.063056,9.959,0.75979
2022-04-07,1.089906,27.7077,87.67,336.58,43469.8,0.99991,0.146052,20.4139,3229.00,15.76737,114.11,103.59,1.5123,0.000025,118.54,0.064429,10.452,0.78705
2022-04-08,1.025486,26.8784,83.83,323.16,42282.6,0.99993,0.142546,19.4176,3193.91,15.13765,110.76,94.58,1.4335,0.000024,109.84,0.062571,9.738,0.75525


In [9]:
# Market capitalization data in trillions of dollars, sourced from Trading View on April 6, 2022.

faangmula_caps_list = [2.804, 0.105855, 1.616, 0.60781, 1.81, 0.012576, 2.245, 0.163534, 0.065299]
faangmula_cap_portfolio = pd.DataFrame({'tickers':faangmula_tickers, 'market cap weights':[market_cap / sum(faangmula_caps_list) for market_cap in faangmula_caps_list]})
faangmula_cap_portfolio

,tickers,market cap weights
0,AAPL,0.297347
1,ABNB,0.011225
2,AMZN,0.171367
3,FB,0.064454
4,GOOG,0.191939
5,LYFT,0.001334
6,MSFT,0.238068
7,NFLX,0.017342
8,UBER,0.006925


In [10]:
mcw_faangmula_daily_returns = pd.DataFrame(faangmula.pct_change().dot(list(faangmula_cap_portfolio.loc[:, 'market cap weights'])))
mcw_faangmula_daily_returns.columns = ['faangmula']
mcw_faangmula_daily_returns

,faangmula
2015-12-01,NaN
2015-12-02,NaN
2015-12-03,NaN
2015-12-04,NaN
2015-12-07,NaN
...,...
2022-04-04,0.024182
2022-04-05,-0.018331
2022-04-06,-0.029120
2022-04-07,-0.000202


In [11]:
random_crypto_portfolios = random_portfolios_generator(crypto_tickers, 10)
random_crypto_portfolios

,ticker,random weights 1,random weights 2,random weights 3,random weights 4,random weights 5,random weights 6,random weights 7,random weights 8,random weights 9,random weights 10
0,BTC,0.034151,0.098408,0.089518,0.071428,0.083555,0.088111,0.054283,0.086440,0.046907,0.069079
1,ETH,0.007662,0.096425,0.016438,0.011175,0.014156,0.006238,0.103564,0.029432,0.076275,0.135465
2,LUNA,0.106659,0.039683,0.022136,0.048173,0.008288,0.097063,0.058448,0.014719,0.087756,0.092829
3,SOL,0.033138,0.092239,0.028884,0.004691,0.077007,0.047486,0.079472,0.008015,0.022656,0.037654
4,XRP,0.022739,0.060297,0.073797,0.033919,0.030399,0.011443,0.046430,0.017375,0.017310,0.041524
5,ADA,0.034251,0.030306,0.050074,0.069735,0.072140,0.112664,0.026530,0.077836,0.086598,0.008547
6,AVAX,0.063487,0.036488,0.086955,0.044296,0.012382,0.032991,0.035717,0.050744,0.092635,0.085527
7,DOT,0.051490,0.048478,0.037033,0.076548,0.098289,0.098824,0.018934,0.069224,0.042532,0.005880
8,DOGE,0.099090,0.089927,0.083035,0.002077,0.048676,0.029525,0.003205,0.087603,0.075313,0.012042
9,SHIB,0.090539,0.001716,0.065678,0.094318,0.045702,0.092617,0.022033,0.084873,0.032428,0.124520


In [12]:
crypto_daily_returns_list = []
for n in range(0, (len(random_crypto_portfolios.columns) - 1)):
        crypto_daily_returns_list.append(crypto.pct_change().dot(list(random_crypto_portfolios.loc[:, f'random weights {n + 1}'])))

crypto_daily_returns = pd.concat(crypto_daily_returns_list, axis = 'columns', join = 'inner')

for n in range(0, (len(random_crypto_portfolios.columns) - 1)):
        crypto_daily_returns = crypto_daily_returns.rename(columns = {n: f'random crypto portfolio {n + 1}'})

crypto_daily_returns

,random crypto portfolio 1,random crypto portfolio 2,random crypto portfolio 3,random crypto portfolio 4,random crypto portfolio 5,random crypto portfolio 6,random crypto portfolio 7,random crypto portfolio 8,random crypto portfolio 9,random crypto portfolio 10
2020-04-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
2022-04-05,-0.016396,-0.019960,-0.007827,-0.015707,-0.022958,-0.024123,-0.019102,-0.015394,-0.011171,-0.011765
2022-04-06,-0.086148,-0.085447,-0.086190,-0.083769,-0.081953,-0.082302,-0.088978,-0.083968,-0.088108,-0.090596
2022-04-07,0.024460,0.024368,0.019409,0.022558,0.021590,0.028063,0.028689,0.020831,0.025551,0.023989
2022-04-08,-0.041301,-0.039294,-0.041617,-0.045172,-0.045125,-0.040697,-0.043668,-0.042354,-0.039874,-0.043022


In [13]:
daily_returns = pd.concat([mcw_faangmula_daily_returns, crypto_daily_returns], axis = 1, join = 'inner')
daily_returns

,faangmula,random crypto portfolio 1,random crypto portfolio 2,random crypto portfolio 3,random crypto portfolio 4,random crypto portfolio 5,random crypto portfolio 6,random crypto portfolio 7,random crypto portfolio 8,random crypto portfolio 9,random crypto portfolio 10
2020-04-20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-04-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2022-04-04,0.024182,-0.012108,-0.010752,-0.006570,-0.011642,-0.010512,-0.014127,-0.016798,-0.009727,-0.012355,-0.012258
2022-04-05,-0.018331,-0.016396,-0.019960,-0.007827,-0.015707,-0.022958,-0.024123,-0.019102,-0.015394,-0.011171,-0.011765
2022-04-06,-0.029120,-0.086148,-0.085447,-0.086190,-0.083769,-0.081953,-0.082302,-0.088978,-0.083968,-0.088108,-0.090596
2022-04-07,-0.000202,0.024460,0.024368,0.019409,0.022558,0.021590,0.028063,0.028689,0.020831,0.025551,0.023989
